In [ ]:
from IPython import get_ipython
from IPython.display import display
from google.colab import files
import pandas as pd
import numpy as np

In [ ]:
print("Vui lòng tải file:")
uploaded = files.upload()

Vui lòng tải file:


Saving DataTroNew.csv to DataTroNew.csv


In [ ]:
def load_and_clean_data(file_path):
    """Tải và làm sạch dữ liệu từ file CSV."""
    df = pd.read_csv(file_path)
    if 'Quyet dinh thue' in df.columns:
        df['Quyet dinh thue'] = df['Quyet dinh thue'].map({'co': 'Yes', 'khong': 'No'})
    else:
        print("Cảnh báo: Cột 'Quyet dinh thue' không tồn tại trong DataFrame.")
    df = df.dropna()
    return df
# Tải dữ liệu
df = load_and_clean_data('DataTroNew.csv')
print("Dữ liệu đã tải:")
print(df.head())

Dữ liệu đã tải:
            Phong Chung chu Dieu hoa Nong lanh             Quat Tu lanh  \
0        khep kin        co       co        co        quat tran      co   
1        khep kin        co    khong        co        quat tran   khong   
2  khong khep kin        co       co     khong         quat cay   khong   
3        khep kin     khong    khong        co  quat treo tuong   khong   
4  khong khep kin     khong       co     khong            khong   khong   

     Bep Tu quan ao      Giuong Gac xep May giat Khoang cach(km) An ninh  \
0     co      tu go  giuong doi      co       co              ~1     tot   
1  khong      tu go  giuong don      co    khong              ~1      on   
2  khong    tu nhua  giuong don   khong    khong            ~0.5      on   
3  khong    tu nhua  giuong don   khong    khong            ~1.5     tot   
4  khong     tu vai  giuong don   khong    khong            ~0.5      on   

  Dien tich Tien dien(nghin/kwh) Tien nuoc/m3 Tien rac/thang Gia phong  \
0 

In [ ]:

def find_entropy(df, target_attribute):
    """Tính entropy của tập dữ liệu dựa trên thuộc tính mục tiêu."""
    target_values = df[target_attribute].unique()
    entropy = 0
    for value in target_values:
        prob = len(df[df[target_attribute] == value]) / len(df)
        if prob > 0:
            entropy -= prob * np.log2(prob)
    return entropy

def find_entropy_attribute(df, attribute, target_attribute):
    """Tính entropy trung bình của một thuộc tính."""
    attribute_values = df[attribute].unique()
    entropy = 0
    for value in attribute_values:
        subset = df[df[attribute] == value]
        prob = len(subset) / len(df)
        entropy += prob * find_entropy(subset, target_attribute)
    return entropy

In [ ]:

def find_information_gain(df, attribute, target_attribute):
    """Tính độ lợi thông tin của một thuộc tính."""
    total_entropy = find_entropy(df, target_attribute)
    attribute_entropy = find_entropy_attribute(df, attribute, target_attribute)
    return total_entropy - attribute_entropy

In [ ]:
def id3(df, features, target_attribute, parent_node_class=None):
    """Xây dựng cây quyết định ID3."""

    if len(df[target_attribute].unique()) == 1:
        return df[target_attribute].iloc[0]

    # Nếu không còn thuộc tính, trả về nhãn phổ biến nhất
    if len(features) == 0:
        return df[target_attribute].mode()[0] if not df[target_attribute].mode().empty else parent_node_class

    # Tìm thuộc tính có information gain cao nhất
    gains = {attr: find_information_gain(df, attr, target_attribute) for attr in features}
    best_attribute = max(gains, key=gains.get)

    # Tạo node gốc với thuộc tính tốt nhất
    tree = {best_attribute: {}}
    remaining_features = [f for f in features if f != best_attribute]

    # Phân nhánh cho từng giá trị của thuộc tính
    for value in df[best_attribute].unique():
        subset = df[df[best_attribute] == value]
        if subset.empty:
            tree[best_attribute][value] = df[target_attribute].mode()[0] if not df[target_attribute].mode().empty else parent_node_class
        else:
            tree[best_attribute][value] = id3(subset, remaining_features, target_attribute, df[target_attribute].mode()[0])

    return tree

In [ ]:
def predict(tree, instance):
    """Dự đoán nhãn dựa trên cây quyết định và một mẫu dữ liệu."""
    if not isinstance(tree, dict):
        return tree
    attribute = list(tree.keys())[0]
    value = instance.get(attribute)
    # Xử lý trường hợp giá trị thuộc tính không có trong cây
    if value not in tree[attribute]:
        print(f"Cảnh báo: Giá trị '{value}' cho thuộc tính '{attribute}' không có trong cây quyết định.")
        return None
    return predict(tree[attribute][value], instance)

In [ ]:
import pandas as pd
import numpy as np


def get_valid_values(df, column):
    """Trả về danh sách giá trị hợp lệ và giá trị phổ biến nhất."""
    # Đảm bảo cột tồn tại trước khi truy cập
    if column not in df.columns:
        print(f"Cảnh báo: Cột '{column}' không tồn tại trong DataFrame.")
        return [], None # Trả về danh sách rỗng và giá trị mặc định là None

    values = df[column].dropna().unique().tolist()
    if not values:
        return [], None

    most_common = df[column].mode()[0] if not df[column].mode().empty else values[0]
    return values, most_common

def interactive_predict(tree, df, features):
    """Dự đoán quyết định thuê trọ với giao diện thân thiện."""
    print("=" * 50)
    print("HỆ THỐNG DỰ ĐOÁN QUYẾT ĐỊNH THUÊ TRỌ")
    print("=" * 50)
    instance = {}
    for feature in features:
        valid_values, default_value = get_valid_values(df, feature)
        print(f"\nChọn giá trị cho thuộc tính '{feature}':")
        for i, value in enumerate(valid_values, start=1):
            print(f"  {i}. {value}")
        if not valid_values and default_value is None:
             print(f"Bỏ qua thuộc tính '{feature}' vì không tìm thấy dữ liệu hợp lệ.")
             continue
        while True:
            user_input = input(f"Nhập số (1-{len(valid_values)}) hoặc nhấn Enter: ").strip()
            try:
                choice = int(user_input)
                if 1 <= choice <= len(valid_values):
                    instance[feature] = valid_values[choice - 1]
                    break
                else:
                    print(f"Vui lòng nhập số từ 1 đến {len(valid_values)}!")
            except ValueError:
                print("Vui lòng nhập số hợp lệ hoặc nhấn Enter!")
        print(f"Đã chọn: {instance[feature]}")
    print("\n" + "=" * 50)
    print("THÔNG TIN ĐÃ NHẬP:")
    for feature, value in instance.items():
        print(f"  {feature}: {value}")
    prediction = predict(decision_tree, instance)
    if prediction is None:
        print("\nKẾT QUẢ DỰ ĐOÁN:")
        print("Không thể dự đoán: Giá trị thuộc tính không hợp lệ hoặc không có trong cây quyết định.")
    else:
        result = 'Có' if prediction == 'Yes' else 'Không'
        print("\nKẾT QUẢ DỰ ĐOÁN:")
        print(f"Quyết định thuê trọ: {result}")

    print("=" * 50)
    return prediction
if 'Quyet dinh thue' in df.columns:
    features = [col for col in df.columns if col != 'Quyet dinh thue']
    target_attribute = 'Quyet dinh thue'
    if not df.empty:
        decision_tree = id3(df, features, target_attribute)
        print(decision_tree)
        interactive_predict(decision_tree, df, features)
    else:
        print("Không có dữ liệu sau khi làm sạch. Không thể xây dựng cây quyết định.")
else:
    print("Không tìm thấy cột 'Quyet dinh thue' trong dữ liệu. Không thể xây dựng cây quyết định.")

{'Gia phong': {'3tr8': 'Yes', '2tr5': {'Quat': {'quat tran': 'Yes', 'quat cay': 'No', 'khong': 'Yes'}}, '1tr': {'Tu quan ao': {'tu nhua': 'No', 'khong': 'No', 'tu go': 'Yes', 'tu vai': 'No'}}, '700k': {'Tu quan ao': {'tu vai': 'Yes', 'khong': 'No'}}, '3tr5': {'Chung chu': {'khong': 'Yes', 'co': 'No'}}, '1tr2': {'Quat': {'quat tran': 'Yes', 'khong': 'No', 'quat cay': 'No'}}, '2tr': {'Tu lanh': {'khong': 'Yes', 'co': 'No'}}, '2tr8': {'Chung chu': {'khong': 'Yes', 'co': 'No'}}, '800k': {'Dieu hoa': {'co': 'Yes', 'khong': 'No'}}, '4tr': {'Chung chu': {'khong': 'No', 'co': 'Yes'}}, '3tr6': {'Quat': {'quat treo tuong': 'Yes', 'quat cay': 'No'}}, '1tr8': 'No', '2tr2': {'Chung chu': {'co': 'Yes', 'khong': 'No'}}, '1tr5': {'Phong': {'khong khep kin': 'No', 'khep kin': 'Yes'}}, '4tr5': {'Quat': {'quat cay': 'No', 'quat tran': 'Yes'}}, '3tr2': 'Yes', '600k': 'Yes'}}
HỆ THỐNG DỰ ĐOÁN QUYẾT ĐỊNH THUÊ TRỌ

Chọn giá trị cho thuộc tính 'Phong':
  1. khep kin
  2. khong khep kin
Nhập số (1-2) hoặc nhấn